In [1]:
import os
import pyspark
import pymssql
import pandas as pd
from pyspark import SparkContext, SparkConf, pandas as ps
from pyspark.sql import SparkSession


In [ ]:
#sparkClassPath = os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.4.1 pyspark-shell'


In [2]:
sparkClassPath =  '/usr/local/spark/jars/postgresql-42.4.1.jar'


In [3]:
try:
    spark.stop()
except:
    print("No Spark Session")
    
spark = SparkSession.builder \
        .config("spark.driver.extraClassPath", sparkClassPath) \
        .config("spark.jars", sparkClassPath) \
        .appName("Southbridge Analytics") \
        .master("local") \
        .getOrCreate()


No Spark Session
22/08/10 19:52:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/10 19:52:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
url = "jdbc:postgresql:pg"
user = "postgres"
password = "P@ssw0rd"
table = "dvdrental.film"

df = spark.read.format("jdbc") \
        .option("url", "jdbc:postgresql://pg:5432/dvdrental") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", "film") \
        .option("user", "postgres").option("password", "P@ssw0rd").load()


In [ ]:
df.show()